# مدل سازی بدون استفاده از ویژگی ها اضافی

## وارد کردن کتابخانه ها

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import h2o
from h2o.estimators.random_forest import H2ORandomForestEstimator
from h2o.grid.grid_search import H2OGridSearch
import warnings

warnings.filterwarnings('ignore')

## استارت h2o

In [5]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM (build 17.0.6+9-LTS-190, mixed mode, sharing)
  Starting server from D:\Projects\LifeExpectancy\venv\Lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\amira\AppData\Local\Temp\tmp0a815o5d
  JVM stdout: C:\Users\amira\AppData\Local\Temp\tmp0a815o5d\h2o_amira_started_from_python.out
  JVM stderr: C:\Users\amira\AppData\Local\Temp\tmp0a815o5d\h2o_amira_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,06 secs
H2O_cluster_timezone:,Asia/Tehran
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.40.0.1
H2O_cluster_version_age:,10 days
H2O_cluster_name:,H2O_from_python_amira_c6aygo
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,1.975 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


## خواندن دیتا

In [6]:
data = h2o.import_file('../../data/ML_Data/Data_2031.csv')

data = data.drop(['ISO3 Alpha-code', 'GDP',
                  'Educated_Population', 'Suicide', 'HIV', 'BMI', 'darmani*',
                  'Estimated road traffic death rate (per 100 000 population)',
                  'Hepatitis B (HepB3) immunization coverage among 1-year-olds (%)'])
data

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


Country,subregion,region,Year,Births (thousands),"Population Density, as of 1 July (persons per square km)","Population Sex Ratio, as of 1 July (males per 100 females)",Population Growth Rate (percentage),"Rate of Natural Change (per 1,000 population)","Crude Birth Rate (births per 1,000 population)",Mean Age Childbearing (years),Sex Ratio at Birth (males per 100 female births),"Crude Death Rate (deaths per 1,000 population)","Under-Five Deaths, under age 5 (thousands)",Net Number of Migrants (thousands),"Net Migration Rate (per 1,000 population)","Life Expectancy at Birth, both sexes (years)"
Burundi,Eastern Africa,AFRICA,1950,116.224,86.864,91.947,2.2,27.788,51.334,30.995,102.5,23.546,29.001,-13.343,-5.893,40.579
Burundi,Eastern Africa,AFRICA,1951,117.067,88.757,92.145,2.114,26.799,50.678,30.996,102.5,23.879,30.669,-13.217,-5.721,40.812
Burundi,Eastern Africa,AFRICA,1952,117.77,90.618,92.319,2.036,26.118,49.933,31.026,102.5,23.815,31.272,-13.715,-5.815,41.003
Burundi,Eastern Africa,AFRICA,1953,119.013,92.451,92.488,1.969,25.844,49.448,31.03,102.5,23.604,31.644,-14.962,-6.216,41.282
Burundi,Eastern Africa,AFRICA,1954,119.972,94.287,92.65,1.965,25.536,48.883,31.036,102.5,23.347,31.915,-14.599,-5.948,41.558
Burundi,Eastern Africa,AFRICA,1955,121.13,96.154,92.806,1.956,25.36,48.399,31.063,102.5,23.039,31.984,-14.669,-5.861,41.855
Burundi,Eastern Africa,AFRICA,1956,122.196,98.045,92.956,1.939,25.101,47.885,31.073,102.5,22.784,31.908,-14.733,-5.773,42.059
Burundi,Eastern Africa,AFRICA,1957,123.309,99.955,93.106,1.921,24.93,47.398,31.086,102.5,22.468,31.831,-15.021,-5.774,42.329
Burundi,Eastern Africa,AFRICA,1958,124.674,101.856,93.25,1.846,24.828,47.013,31.125,102.5,22.185,31.769,-17.047,-6.428,42.559
Burundi,Eastern Africa,AFRICA,1959,126.251,103.801,93.392,1.937,24.801,46.736,31.151,102.5,21.935,31.804,-14.813,-5.483,42.779


## جدا سازی داده های تست و ولیدیشن و آموزش

In [7]:
test = data[data["Year"] > 2021]
train_val_data = data[data["Year"] <= 2021]

x = train_val_data.columns
y = 'Life Expectancy at Birth, both sexes (years)'
x.remove(y)

ss = data.split_frame(seed=1)
train = ss[0]
valid = ss[1]

## مشخص کردن هایپر پارامترها

In [8]:
drf_params1 = {'ntrees': [50, 75, 100],
             'max_depth': [5, 10, 15],
             'min_rows': [1, 5]}

drf_grid1 = H2OGridSearch(model=H2ORandomForestEstimator,
                          grid_id='drf_grid1',
                          hyper_params=drf_params1)

## آموزش مدل

In [9]:
drf_grid1.train(x=x, y=y,
                training_frame=train,
                validation_frame=valid,
                seed=1)

drf Grid Build progress: |███████████████████████████████████████████████████████| (done) 100%


,max_depth,min_rows,ntrees,model_ids,residual_deviance
,15.0,1.0,100.0,drf_grid1_model_15,0.2718317
,15.0,1.0,75.0,drf_grid1_model_9,0.2730259
,15.0,1.0,50.0,drf_grid1_model_3,0.2758856
,15.0,5.0,100.0,drf_grid1_model_18,0.4535914
,15.0,5.0,75.0,drf_grid1_model_12,0.4550803
,15.0,5.0,50.0,drf_grid1_model_6,0.4613844
,10.0,1.0,100.0,drf_grid1_model_14,0.7181298
,10.0,1.0,50.0,drf_grid1_model_2,0.7213876
,10.0,1.0,75.0,drf_grid1_model_8,0.7215663
,10.0,5.0,75.0,drf_grid1_model_11,0.7894335


## مرتب کردن مدل ها بر اساس R^2

In [10]:
drf_gridperf1 = drf_grid1.get_grid(sort_by='r2', decreasing=True)
display(drf_gridperf1)


,max_depth,min_rows,ntrees,model_ids,r2
,15.0,1.0,100.0,drf_grid1_model_15,0.9979810
,15.0,1.0,75.0,drf_grid1_model_9,0.9979721
,15.0,1.0,50.0,drf_grid1_model_3,0.9979509
,15.0,5.0,100.0,drf_grid1_model_18,0.9966310
,15.0,5.0,75.0,drf_grid1_model_12,0.9966199
,15.0,5.0,50.0,drf_grid1_model_6,0.9965731
,10.0,1.0,100.0,drf_grid1_model_14,0.9946662
,10.0,1.0,50.0,drf_grid1_model_2,0.9946420
,10.0,1.0,75.0,drf_grid1_model_8,0.9946407
,10.0,5.0,75.0,drf_grid1_model_11,0.9941366


## انتخاب بهترین مدل

In [11]:
best_drf1 = drf_gridperf1.models[0]
best_drf1.summary()

,number_of_trees,number_of_internal_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
,100.0,100.0,8368194.0,15.0,15.0,15.0,4743.0,6383.0,5637.73


In [12]:
best_drf1.model_performance(train).r2()

0.9994327715441402

## ارزیابی مدل

In [13]:
best_drf_perf1 = best_drf1.model_performance(test)

best_drf_perf1.r2()

0.998292876227587

In [14]:
pred_test = best_drf1.predict(test)
pred_test

drf prediction progress: |███████████████████████████████████████████████████████| (done) 100%


predict
62.235
62.6768
63.3892
63.6844
63.6619
63.7978
63.9811
64.1734
64.2357
64.3408
